In [10]:
!pip install datasets

In [1]:
!pip install opencv-python
!pip install torch torchvision pillow numpy

   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
    --------------------------------------- 0.5/39.5 MB 1.2 MB/s eta 0:00:33
   - -------------------------------------- 1.0/39.5 MB 1.3 MB/s eta 0:00:30
   - -------------------------------------- 1.0/39.5 MB 1.3 MB/s eta 0:00:30
   - -------------------------------------- 1.3/39.5 MB 1.2 MB/s eta 0:00:33
   - -------------------------------------- 1.6/39.5 MB 1.1 MB/s eta 0:00:34
   - -------------------------------------- 1.8/39.5 MB 1.2 MB/s eta 0:00:31
   -- ------------------------------------- 2.1/39.5 MB 1.2 MB/s eta 0:00:31
   -- ------------------------------------- 2.4/39.5 MB 1.2 MB/s eta 0:00:31
   -- ------------------------------------- 2.6/39.5 MB 1.2 MB/s eta 0:00:32
   -- ------------------------------------- 2.9/39.5 MB 1.2 MB/s eta 0:00:31
   --- -------------

In [3]:
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'fcn_resnet50', pretrained=True)
# or
# model = torch.hub.load('pytorch/vision:v0.10.0', 'fcn_resnet101', pretrained=True)
model.eval()

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to C:\Users\Musab Bin Abaid/.cache\torch\hub\v0.10.0.zip
c:\Users\Musab Bin Abaid\MS-thesis\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Musab Bin Abaid\MS-thesis\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FCN_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=FCN_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fcn_resnet50_coco-1167a1af.pth" to C:\Users\Musab Bin Abaid/.cache\torch\hub\checkpoints\fcn_resnet50_coco-1167a1af.pth
100.0%


FCN(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
# import segmentation_models_pytorch as smp
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets, transforms, models


# ---- Model Selection ----
# def get_model(model_name):
#     if model_name == "DeepLabV3+":
#         return DeepLabV3Plus(encoder_name="resnet50", in_channels=3, classes=1)
#     elif model_name == "U-Net":
#         return Unet(encoder_name="resnet50", in_channels=3, classes=1)
#     elif model_name == "MFFENet":
#         from mffenet import MFFENet  # Assume a custom implementation
#         return MFFENet(backbone="resnet50", num_classes=1)
#     elif model_name == "FCN":
#         fcn = torch.hub.load('pytorch/vision:v0.10.0', 'fcn_resnet50', pretrained=True)
#         return fcn
#     else:
#         raise ValueError("Unknown model name")

# ---- Hyperparameters ----
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 8
learning_rate = 0.01
momentum = 0.9
weight_decay = 0.0005
num_epochs = 50

# ---- Data Preprocessing ----
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# train_dataset = CustomLandslideDataset(root="atrain_data", transform=transform)
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

dataset = datasets.ImageFolder(root=r'C:\Users\Musab Bin Abaid\MS-thesis\atrain_data', transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)
# ---- Training Function ----
def train(model):
    # model = get_model(model_name).to(device)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)
    criterion = torch.nn.CrossEntropyLoss()  # Dice loss for imbalanced segmentation

    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)

            optimizer.zero_grad()
            outputs =  model(images)
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

train(model)
# # ---- Train the models ---
# for model_name in ["DeepLabV3+", "U-Net", "MFFENet", "FCN"]:
#     print(f"\nTraining {model_name}...")
#     train(model_name)


TypeError: cross_entropy_loss(): argument 'input' (position 1) must be Tensor, not collections.OrderedDict

In [ ]:
import os
import numpy as np
from PIL import Image

def create_directories(output_dir):
    landslide_img_dir = os.path.join(output_dir, "landslide", "images")
    landslide_mask_dir = os.path.join(output_dir, "landslide", "masks")
    non_landslide_img_dir = os.path.join(output_dir, "non-landslide", "images")
    non_landslide_mask_dir = os.path.join(output_dir, "non-landslide", "masks")
    
    os.makedirs(landslide_img_dir, exist_ok=True)
    os.makedirs(landslide_mask_dir, exist_ok=True)
    os.makedirs(non_landslide_img_dir, exist_ok=True)
    os.makedirs(non_landslide_mask_dir, exist_ok=True)
    
    return landslide_img_dir, landslide_mask_dir, non_landslide_img_dir, non_landslide_mask_dir

def process_images(image_folder, mask_folder, output_folder, threshold=5000):
    landslide_img_dir, landslide_mask_dir, non_landslide_img_dir, non_landslide_mask_dir = create_directories(output_folder)
    
    image_files = [f for f in os.listdir(image_folder) if f.endswith(".png")]
    print(f"Processing {len(image_files)} images...")
    
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        mask_path_default = os.path.join(mask_folder, image_file)
        mask_path_tif = os.path.join(mask_folder, os.path.splitext(image_file)[0] + ".png")

        # Check if either mask exists
        mask_path_default = os.path.join(mask_folder, image_file)
        mask_path_tif = os.path.join(mask_folder, os.path.splitext(image_file)[0] + ".png")

        # Assign the correct mask path if it exists
        if os.path.exists(mask_path_default):
            mask_path = mask_path_default
        elif os.path.exists(mask_path_tif):
            mask_path = mask_path_tif
        else:
            print(f"Mask not found for {image_file}, skipping...")
            continue

        # Load mask and compute landslide area
        mask = Image.open(mask_path).convert("L")
        mask_array = np.array(mask)
        landslide_area = np.sum(mask_array==0)  # Count non-zero pixels
        
        # Determine category and save image & mask
        image_name = os.path.splitext(image_file)[0] + ".png"
        mask_name = os.path.splitext(image_file)[0] + ".png"
        
        if landslide_area >= threshold:
            image_save_path = os.path.join(landslide_img_dir, image_name)
            mask_save_path = os.path.join(landslide_mask_dir, mask_name)
            category = "landslide_wenchan"
        else:
            continue
            # image_save_path = os.path.join(non_landslide_img_dir, image_name)
            # mask_save_path = os.path.join(non_landslide_mask_dir, mask_name)
            # category = "non-landslidewenchan"
        
        # Save image and mask
        image = Image.open(image_path).convert("RGB")
        image.save(image_save_path, "PNG")
        mask.save(mask_save_path, "PNG")
        
        print(f"Saved {image_file} as PNG to {category} folder")

# Example usage
image_folder = "/home/iml/Downloads/Remote_sensing/train_data/landslide_images"
mask_folder = "/home/iml/Downloads/Remote_sensing/landslide_images_masks"
output_folder = "/home/iml/Downloads/Remote_sensing/filtered_images"
thresh = 6000  # Adjust threshold as needed

process_images(image_folder, mask_folder, output_folder, threshold=thresh)


Processing 5079 images...
Saved Hokkaido0134.png as PNG to landslide_wenchan folder
Saved LongxiheSAT1375.png as PNG to landslide_wenchan folder
Saved LongxiheSAT1186.png as PNG to landslide_wenchan folder
Saved LongxiheSAT0504.png as PNG to landslide_wenchan folder
Saved LongxiheSAT0062.png as PNG to landslide_wenchan folder
Saved LongxiheSAT1209.png as PNG to landslide_wenchan folder
Saved jiuzzhaigou_0.5m_UVA1617.png as PNG to landslide_wenchan folder
Saved jiuzzhaigou_0.5m_UVA0004.png as PNG to landslide_wenchan folder
Saved Hokkaido0133.png as PNG to landslide_wenchan folder
Saved Lombok202.png as PNG to landslide_wenchan folder
Saved LongxiheSAT1628.png as PNG to landslide_wenchan folder
Saved jiuzzhaigou_0.5m_UVA1033.png as PNG to landslide_wenchan folder
Saved Hokkaido0480.png as PNG to landslide_wenchan folder
Saved Hokkaido0638.png as PNG to landslide_wenchan folder
Saved Hokkaido1524.png as PNG to landslide_wenchan folder
Saved jiuzzhaigou_0.5m_UVA0970.png as PNG to landslid

In [7]:
import os
import glob

directory = "/home/iml/Downloads/Remote_sensing/non_landslide_testing_data"
png_files = glob.glob(os.path.join(directory, "*.png"))

print(f"Number of PNG files: {len(png_files)}")


Number of PNG files: 128


In [2]:
!pip install torch torchvision pillow numpy

  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached fsspec-2025.2.0-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse

In [8]:
import torch
from torchvision import datasets, transforms, models

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize(           # Normalize with ImageNet stats
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Load the dataset
dataset = datasets.ImageFolder(root='/home/iml/Downloads/Remote_sensing/train_data', transform=transform)

# Split the dataset into training and validation sets
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)
# Load a pre-trained ResNet model
model = models.resnet18(pretrained=True)

# Modify the final layer for binary classification
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 2)  # 2 output classes: landslide and non-landslide

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        print("current loss is this : ",loss.item())
        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%")

/home/iml/Downloads/Remote_sensing/.conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/iml/Downloads/Remote_sensing/.conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


current loss is this :  0.7719568610191345
current loss is this :  1.074949026107788
current loss is this :  2.4205682277679443
current loss is this :  1.2264447212219238
current loss is this :  0.996244490146637
current loss is this :  0.7157060503959656
current loss is this :  0.8227885365486145
current loss is this :  0.8270589709281921
current loss is this :  0.487691730260849
current loss is this :  0.7866650819778442
current loss is this :  0.7494245171546936
current loss is this :  0.6675268411636353
current loss is this :  0.470289409160614
current loss is this :  0.463142454624176
current loss is this :  0.3810490071773529
current loss is this :  0.4463713765144348
current loss is this :  0.7545108795166016
current loss is this :  0.44666022062301636
current loss is this :  0.5848295092582703
current loss is this :  0.542803943157196
current loss is this :  0.7503843307495117
Epoch 1, Loss: 0.7803364850225902
Validation Loss: 1.271734709541003, Accuracy: 59.523809523809526%
cu

In [6]:
torch.save(model.state_dict(), 'landslide_classifier.pth')

In [9]:
# Load the saved model
model.load_state_dict(torch.load('landslide_classifier.pth'))
model.eval()

# Test on a single image
from PIL import Image

def predict_image(image_path):
    image = Image.open(image_path)
    image = transform(image).unsqueeze(0).to(device)
    output = model(image)
    _, predicted = torch.max(output, 1)
    return 'Landslide' if predicted.item() == 0 else 'Non-Landslide'

# Example usage
print(predict_image('/home/iml/Downloads/Remote_sensing/train_data/landslide_images/Hokkaido0030.png'))

Landslide


# Seleting random 500 images with diversity from dataset

In [4]:
import os
import shutil
import random
from collections import defaultdict

# Define paths

source_folder = "/home/iml/Downloads/Remote_sensing/train_data/non_landslide_images"
destination_folder = "/home/iml/Downloads/Remote_sensing/train_data/non_landslide_images"
os.makedirs(destination_folder, exist_ok=True)

# Read images and categorize based on initial names
class_images = defaultdict(list)

for filename in os.listdir(source_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Filter image files
        class_name = filename[0:5]  # Assuming class is determined by initial name before '_'
        class_images[class_name].append(filename)

# Ensure at least one image per class
selected_images = []
num_required = 100

# Shuffle and distribute images
all_images = []
for cls, images in class_images.items():
    random.shuffle(images)
    all_images.extend(images)

random.shuffle(all_images)

# Select up to 500 images while maintaining class diversity
selected_images = all_images[:num_required]

# Copy selected images to new folder
for img in selected_images:
    shutil.move(os.path.join(source_folder, img), os.path.join(destination_folder, img))

print(f"Successfully copied {len(selected_images)} images to {destination_folder}")


Successfully copied 100 images to /home/iml/Downloads/Remote_sensing/testing_data


In [6]:
import os
import shutil

testing_data_dir = '/home/iml/Downloads/Remote_sensing/testing_data'
filtered_images_dir = '/home/iml/Downloads/Remote_sensing/filtered_images/landslide/images'
destination_dir = '/home/iml/Downloads/Remote_sensing/testing_landslide_images'

# Get the list of images in the testing_data directory
testing_images = os.listdir(testing_data_dir)

# Get the list of images in the filtered_images directory
filtered_images = os.listdir(filtered_images_dir)

# Find the common images in both directories
common_images = set(testing_images) & set(filtered_images)

# Move the common images to the destination directory
for image in common_images:
    image_path = os.path.join(testing_data_dir, image)
    destination_path = os.path.join(destination_dir, image)
    shutil.move(image_path, destination_path)

# Return the names of the common images
common_image_names = list(common_images)
common_image_names

['LongxiheSAT0324.png',
 'LongxiheSAT0719.png',
 'LongxiheSAT0827.png',
 'jiuzzhaigou_0.5m_UVA0889.png',
 'jiuzzhaigou_0.5m_UVA1486.png',
 'LongxiheSAT1191.png',
 'LongxiheSAT0616.png',
 'Hokkaido1112.png',
 'jiuzzhaigou_0.5m_UVA0649.png',
 'jiuzzhaigou_0.5m_UVA1710.png',
 'LongxiheSAT0274.png',
 'jiuzzhaigou_0.5m_UVA1590.png',
 'LongxiheSAT1651.png',
 'Hokkaido0601.png',
 'Hokkaido1523.png',
 'Hokkaido0415.png',
 'Hokkaido0479.png',
 'Hokkaido0693.png',
 'Hokkaido0374.png',
 'Hokkaido0795.png',
 'jiuzzhaigou_0.5m_UVA1744.png',
 'jiuzzhaigou_0.5m_UVA0819.png',
 'LongxiheSAT0975.png',
 'jiuzzhaigou_0.5m_UVA0338.png',
 'LongxiheSAT1626.png',
 'wenchuan146.png',
 'LongxiheSAT0443.png',
 'Hokkaido0254.png',
 'LongxiheSAT0790.png',
 'jiuzzhaigou_0.5m_UVA0486.png',
 'jiuzzhaigou_0.5m_UVA1653.png',
 'LongxiheSAT1734.png',
 'jiuzzhaigou_0.5m_UVA0944.png',
 'jiuzzhaigou_0.5m_UVA1241.png',
 'Hokkaido0652.png',
 'jiuzzhaigou_0.5m_UVA0592.png',
 'LongxiheSAT0654.png',
 'Hokkaido0089.png',
 'Longxi

In [4]:
!pip install h5py
!pip install matplotlib


  Using cached contourpy-1.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.56.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (101 kB)
  Using cached kiwisolver-1.4.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.2 kB)
  Using cached pyparsing-3.2.1-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 5.8 MB/s eta 0:00:00a 0:00:01
Using cached contourpy-1.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (326 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.56.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.9 MB)
Using cached kiwisolver-1.4.8-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.4 MB)
Using cached pyparsing-3.2.1-py3-none-any.whl (107 kB)


In [18]:
!pip install matplotlib

   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.1 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.1 MB 541.6 kB/s eta 0:00:14
   -- ------------------------------------- 0.5/8.1 MB 541.6 kB/s eta 0:00:14
   -- ------------------------------------- 0.5/8.1 MB 541.6 kB/s eta 0:00:14
   --- ------------------------------------ 0.8/8.1 MB 524.3 kB/s eta 0:00:14
   --- ------------------------------------ 0.8/8.1 MB 524.3 kB/s eta 0:00:14
   ----- ---------------------------------- 1.0/8.1 MB 524.3 kB/s eta 0:00:14
   ----- ---------------------------------- 1.0/8.1 MB 524.3 kB/s eta 0:00:14
   ------ --------------------------------- 1.3/8.1 MB 528.5 kB/s eta 0:00:13
   ------ ---------------------

In [26]:
from PIL import Image
import numpy as np
import random
import os

def get_image_size(image_path):
    try:
        with Image.open(image_path) as img:
            width, height = img.size
            print(f"Image Size: {width}x{height} pixels")
            return width, height
    except Exception as e:
        print(f"Error: {e}")
        return None

def extract_non_landslide_patch(image_path, mask_path, output_dir="patches", patch_size=256):
    try:
        with Image.open(image_path) as img, Image.open(mask_path) as mask:
            img = img.convert("RGB")
            mask = mask.convert("L")
            
            img_array = np.array(img)
            mask_array = np.array(mask)
            
            height, width = mask_array.shape
            
            non_landslide_indices = np.argwhere(mask_array == 255)  # White regions (non-landslide)
            
            if len(non_landslide_indices) == 0:
                print(f"No non-landslide regions found in {image_path}.")
                return None
            
            # Ensure output directory exists
            os.makedirs(output_dir, exist_ok=True)
            
            # Randomly select a starting point for the patch
            random.shuffle(non_landslide_indices)
            for y, x in non_landslide_indices:
                if x + patch_size <= width and y + patch_size <= height:
                    patch = img_array[y:y+patch_size, x:x+patch_size]
                    patch_image = Image.fromarray(patch)
                    
                    # Save patch
                    patch_name = os.path.basename(image_path).replace(".png", "_patch.png")
                    patch_path = os.path.join(output_dir, patch_name)
                    patch_image.save(patch_path)
                    print(f"Patch saved at: {patch_path}")
                    return patch_image
            
            print(f"No suitable patch found in {image_path}.")
            return None
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

def process_directory(image_dir, mask_dir, output_dir="patches", patch_size=256):
    os.makedirs(output_dir, exist_ok=True)
    
    image_files = [f for f in os.listdir(image_dir) if f.endswith(".png")]
    
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        mask_path = os.path.join(mask_dir, image_file)
        
        if os.path.exists(mask_path):
            extract_non_landslide_patch(image_path, mask_path, output_dir, patch_size)
        else:
            print(f"Mask not found for {image_file}")

# Example usage
image_dir = r"C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\images"
mask_dir = r"C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\masks"
output_dir = r"C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\patches"

process_directory(image_dir, mask_dir, output_dir)


Patch saved at: C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\patches\Hokkaido0001_patch.png
Patch saved at: C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\patches\Hokkaido0005_patch.png
Patch saved at: C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\patches\Hokkaido0025_patch.png
Patch saved at: C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\patches\Hokkaido0037_patch.png
Patch saved at: C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\patches\Hokkaido0038_patch.png
Patch saved at: C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\patches\Hokkaido0050_patch.png
Patch saved at: C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\patches\Hokkaido0051_patch.png
Patch saved at: C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\patches\Hokkaido0053_patch.png
Patch saved at: C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\patches\Hokkaido0068_patch.png
Patch saved at: C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\patches\Hokkaido0071_patch.png
Patch saved at: C:\Users\Musab Bin Abaid\MS-thesis\atrain_da

In [27]:
from PIL import Image
import numpy as np
import os

def extract_max_landslide_patch(image_path, mask_path, output_dir="patches", patch_size=256):
    try:
        with Image.open(image_path) as img, Image.open(mask_path) as mask:
            img = img.convert("RGB")
            mask = mask.convert("L")
            
            img_array = np.array(img)
            mask_array = np.array(mask)
            
            height, width = mask_array.shape
            
            max_landslide_area = 0
            best_patch = None
            best_patch_coords = None
            
            # Iterate over possible patches
            for y in range(0, height - patch_size, patch_size // 2):
                for x in range(0, width - patch_size, patch_size // 2):
                    patch_mask = mask_array[y:y+patch_size, x:x+patch_size]
                    landslide_area = np.sum(patch_mask == 0)  # Count landslide pixels (black)
                    
                    if landslide_area > max_landslide_area:
                        max_landslide_area = landslide_area
                        best_patch = img_array[y:y+patch_size, x:x+patch_size]
                        best_patch_coords = (x, y)
            
            if best_patch is not None:
                patch_image = Image.fromarray(best_patch)
                os.makedirs(output_dir, exist_ok=True)
                
                patch_name = os.path.basename(image_path).replace(".png", "_max_landslide_patch.png")
                patch_path = os.path.join(output_dir, patch_name)
                patch_image.save(patch_path)
                print(f"Max landslide patch saved at: {patch_path}, Coordinates: {best_patch_coords}")
                return patch_image
            else:
                print(f"No significant landslide patch found in {image_path}.")
                return None
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

def process_directory_max_landslide(image_dir, mask_dir, output_dir="patches", patch_size=256):
    os.makedirs(output_dir, exist_ok=True)
    
    image_files = [f for f in os.listdir(image_dir) if f.endswith(".png")]
    
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        mask_path = os.path.join(mask_dir, image_file)
        
        if os.path.exists(mask_path):
            extract_max_landslide_patch(image_path, mask_path, output_dir, patch_size)
        else:
            print(f"Mask not found for {image_file}")

# Example usage
image_dir = r"C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\images"
mask_dir = r"C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\masks"
output_dir = r"C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\landslide_patches"

process_directory_max_landslide(image_dir, mask_dir, output_dir)


No significant landslide patch found in C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\images\Hokkaido0001.png.
No significant landslide patch found in C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\images\Hokkaido0005.png.
Max landslide patch saved at: C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\landslide_patches\Hokkaido0025_max_landslide_patch.png, Coordinates: (0, 128)
No significant landslide patch found in C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\images\Hokkaido0037.png.
No significant landslide patch found in C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\images\Hokkaido0038.png.
Max landslide patch saved at: C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\landslide_patches\Hokkaido0050_max_landslide_patch.png, Coordinates: (128, 0)
Max landslide patch saved at: C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\landslide_patches\Hokkaido0051_max_landslide_patch.png, Coordinates: (0, 0)
No significant landslide patch found in C:\Users\Musab Bin Abaid\MS-thesis\atrain_data\im